In [24]:
# Importujeme si opět knihovny jako minule
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

# https://pypi.org/project/simplemma/
import simplemma
import pandas as pd
import keras
import tensorflow as tf
from transformers import pipeline
from transformers import logging

# hlásit pouze nejkritičtější chyby
logging.set_verbosity(50)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.18.0
Keras version: 3.6.0


## Word2VEc
- ukážeme si jak vytvořit embedding space pomocí knihovny [Gensim](https://radimrehurek.com/gensim/models/word2vec.html)
- lze to i v Keras – [Embedding layer](https://keras.io/api/layers/core_layers/embedding/)
- využijeme dataset [CIIRC-NLP/czech_news_simple-cs](https://huggingface.co/datasets/CIIRC-NLP/czech_news_simple-cs), který obsahuje články z českých novinových stránek mezi lety 2000-2022
    - obsahuje 200 záznamů pro každou z 5 kategorií (Zahraniční, Domácí, Sport, Kultura, Ekonomika)
    - tento dataset je podmnožina datasetu [hynky/czech_news_dataset_v2](https://huggingface.co/datasets/hynky/czech_news_dataset_v2), který obsahuje přes 1,6 miliónů záznamů

In [25]:
df = pd.read_parquet("hf://datasets/CIIRC-NLP/czech_news_simple-cs/data/test-00000-of-00001-8fd5c2a953da2a24.parquet")
df.head(5)

,url,headline,brief,keywords,category,content,category_unclean
0,https://www.irozhlas.cz/ekonomika/citibank-okd...,Soud potvrdil platnost pohledávek Citibank za ...,"Pohledávky za více než deset miliard korun, kt...","[Okd, Citibank, Pohledávky, Ostrava, Soud, Poh...",5,"V roce 2019 ostravský soud rozhodl, že pohledá...",Ekonomika
1,https://www.novinky.cz/ekonomika/clanek/lego-c...,Lego chce dát vale plastovým obalům. Pilotní p...,"Lego bude balit stavebnice do papíru, nahradí ...",[],5,Na zavádění nového typu balení a s tím souvise...,Ekonomika
2,https://www.idnes.cz/ekonomika/domaci/hypoteka...,"Hypoteční mejdan končí, úvěry podraží a klesne...",Objem poskytnutých hypoték za letošní rok míří...,"[Česká spořitelna, Čnb - česká národní banka, ...",5,Nad touto hranicí byla průměrná úroková sazba ...,Ekonomika
3,https://www.novinky.cz/ekonomika/clanek/k-milo...,K Milostivému létu se přidaly další banky,"Možností, jak se v rámci tzv. Milostivého léta...",[],5,Milostivé léto je novelou exekučního řádu urče...,Ekonomika
4,https://www.idnes.cz/zpravy/zahranicni/vakcina...,V evropských zemích se hromadí vakcína AstraZe...,Zemím Evropské unie se ve skladech hromadí nev...,"[Francie, Itálie, Španělsko, Německo, Evropská...",1,Například Francie do pátku zužitkovala 16 proc...,Zahraničí


In [26]:
# My se zaměříme pouze na sloupec "content", který obsahuje celý text článku
df.info()

# Předtím něž se podíváte jak jsem udělal preprocessing, popřemýšlejte sami, jak byste jej udělali vy :)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   url               1000 non-null   object
 1   headline          1000 non-null   object
 2   brief             1000 non-null   object
 3   keywords          1000 non-null   object
 4   category          1000 non-null   int64 
 5   content           1000 non-null   object
 6   category_unclean  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [ ]:
# Pro odstranění předložek a spojek jsem si vytvořil script, který který z wikipedie vyextrahuje všechny předložky a spojky
# Cílem je odstranit tyto slova z textu, protože nám nepřináší žádnou informaci navíc
# Napadl by někoho lepší způsob? :)
combined_words_df = pd.read_csv('prepositions_conjuctions.csv')
combined_words_df.head()
combined_words_df['word'].values

array(['za', 'před', 'pod', 'nad', 'mezi', 'z', 'z', 'do', 'bez', 'kromě',
       'krom', 'místo', 'podle', 'podél', 'kolem', 'okolo', 'u', 'vedle',
       'během', 'pomocí', 'stran', 'prostřednictvím', 'za', 'vinou',
       'naproti', 'proti', 'oproti', 'kvůli', 'díky', 'vůči', 'za',
       'před', 'mimo', 'na', 'pode', 'pod', 'nade', 'nad', 'mezi',
       'skrze', 'skrz', 'přes', 'o', 'po', 'v', 'na', 'v', 'po', 'při',
       'proto', 'a proto', 'tak', 'a tak', 'tudíž', 'a tudíž', 'tedy',
       'a', 'i', 'ani', 'nebo', 'či', 'přímo', 'nadto', 'ani–ani',
       'jak–tak', 'hned–hned', 'jednak–jednak', 'zčásti–zčásti',
       'dílem–dílem', 'a', 'ale', 'avšak', 'však', 'leč', 'nýbrž',
       'naopak', 'jenomže', 'jenže', 'sice–ale', 'jistě–ale', 'i', 'ba',
       'ba i', 'ba ani', 'nadto', 'dokonce', 'nejen – ale i',
       'nejen – nýbrž i', 'nebo', 'aneb', 'buď–nebo', 'buď–anebo',
       'totiž', 'vždyť', 'neboť', 'vždyť', 'totiž', 'však', 'také', 'aby',
       'jakmile', 'až', 'než

In [47]:
data = df["content"]

# Kontrola, zda neobsahuje nějaké nulové hodnoty
print(f"Number of null values: {data.isnull().sum()}")

# Preprocessing 
# Primárně využívám regex (pro lepší pochopení doporučuji zkusit https://regex101.com)

# Převedení na malá písmena
data = data.str.lower()
# Odstranění čísel
data = data.str.replace(r'\d+', '', regex=True)
# Odstranění předložek a spojek
data = data.apply(lambda x: ' '.join(['' if word in combined_words_df['word'].values else word for word in x.split()]))
# # Odstranění speciálních znaků (kromě tečky, kterou použijeme pro rozdělení na věty)
data = data.str.replace(r'‚|,|(\?)|!', '', regex=True)
# Lemmatizace
# tj. převedení slov na základní tvar (např. "běžel" -> "běžet")
data = data.apply(lambda x: ' '.join([simplemma.lemmatize(word, lang='cs') for word in x.split()]))
# Odstranění diakritiky
data = data.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# Odstranění nadbytečných mezer
data = data.str.replace(r' +', ' ', regex=True)

# Titulky rozdělíme na věty
sentences = data.str.split('.').explode()
# Odstraníme mezery na začátku a na konci, které mohly vzniknout rozdělnením
sentences = sentences.str.strip()

# Rozdělíme věty na slova
word_sentences = sentences.str.split()
word_sentences.reset_index(drop=True, inplace=True)

print(f"Number of sentences: {len(sentences)}")
word_sentences.head(5)

Number of null values: 0
Number of sentences: 24951


0    [rok, ostravsky, soud, rozhodnout, pohledavka,...
1    [tento, rozsudek, se, odvolat, insolvencni, sp...
2    [nyni, se, soud, zabyvat, uz, jen, cast, spor,...
3    [krajsky, soud, nyni, odpurci, zaloba, insolve...
4    [dokazovani, doplneny, znalecke, posudky, dosp...
Name: content, dtype: object

In [29]:
from gensim.models import Word2Vec

# Vytvoření word2vec embeddingu
embedding = Word2Vec(sentences=word_sentences.tolist(), vector_size=100, window=5, min_count=2, workers=4)
# Parametry:
# sentences - data
# vector_size - dimenze embeddingu
# window - okolí pro kontext
# min_count - minimální frekvence slova (pokud je slovo méně časté, tak se ignoruje)
# workers - počet jader pro trénování (tj. využíváme paralelizace)

In [ ]:
print(f"Velikost slovníku: {len(embedding.wv.key_to_index)}")
# Výpis prvních 100 slov v embeddingu
print(list(embedding.wv.key_to_index.keys())[:100])

Velikost slovníku: 19686
['byt', 'se', 'ten', 'ktery', 's', 'on', 'mit', 'rok', 'v', 'k', 'pro', 'ja', 'jeho', 'moci', 'svuj', 'uz', 'od', 'jako', 'tento', 'clovek', 'dalsi', 'jeden', 'co', 'cesky', 'jak', 'vsechen', 'velky', 'hodne', 'uvest', 'muset', 'stat', 'dva', 'jenz', 'jen', 'z', 'novy', 'chtit', 'prvni', 'pak', 'muj', 'procento', 'jeste', 'jit', 'rici', 'zeme', 'kdy', 'doba', 'kde', 'dostat', 'posledni', 'druhy', 'tri', 'jiny', 'napriklad', 'cena', 'vlada', 'spolecnost', 'pripad', 'cely', 'treba', 'situace', 'tam', 'milion', 'dat', 'tym', '-', 'strana', 'zacit', 'nekolik', 'ty', 'mozny', 'rada', 'prave', 'nyni', 'evropsky', 'rikat', 'tady', 'nektery', 'prace', 'zapas', 'firma', 'hrat', 'tyden', 'kazdy', 'coz', 'tisic', ':', 'takovy', 'cast', 'misto', 'hlavni', 'prijit', 'velmi', 'konec', 'asi', 'koruna', 'ted', 'rusky', 'problem', 'mesto']


In [ ]:
# Můžeme získat slova vyskytující se často v okolí slova "cesky"
# tj. slova, která se často vyskytují v kontextu
embedding.wv.most_similar("cesky")

[('uvest', 0.96949303150177),
 ('mluvci', 0.9641792178153992),
 ('reditel', 0.9601395726203918),
 ('diplomacie', 0.9588109254837036),
 ('ministerstvo', 0.9575371742248535),
 ('tiskovy', 0.9574517011642456),
 ('vnitro', 0.9567068219184875),
 ('dmitrij', 0.9557474255561829),
 ('zdravotnictvi', 0.9548031687736511),
 ('statisticky', 0.9547945261001587)]

In [ ]:
# Reprezentace slova "soud" ve vytvořeném embeddingu
embedding.wv['soud']

array([-3.3556578e-01,  6.6056442e-01,  2.9289553e-01,  2.0922945e-01,
        1.3379279e-01, -1.1157229e+00,  7.7918214e-01,  1.5907409e+00,
       -6.6003579e-01, -6.5995681e-01, -7.7565603e-02, -1.1365488e+00,
       -7.9070950e-01,  3.2764357e-01,  1.9667825e-01, -4.3729270e-01,
        2.1123627e-01, -9.5436287e-01, -2.3127641e-01, -8.3508027e-01,
        3.3504006e-01,  2.5344446e-01,  7.6245689e-01, -8.9429356e-02,
       -1.8001343e-01, -1.1773935e-03, -5.7082891e-01, -1.5073353e-01,
       -5.1444244e-01,  2.1846554e-01,  1.0024707e+00, -2.0972893e-01,
        4.4090363e-01, -6.6870922e-01, -2.3017351e-01,  5.8174312e-01,
        2.1814294e-01, -3.3779317e-01, -3.8098174e-01, -7.9261786e-01,
        2.9269329e-01, -4.1061288e-01, -5.6123996e-01,  1.8403904e-01,
        5.4575789e-01, -4.0096694e-01, -3.6220595e-01,  1.1667681e-01,
        6.0313123e-01,  8.8592899e-01,  1.1332589e-01, -3.5279930e-01,
       -1.7052186e-01, -6.0629923e-02, -1.6450121e-01,  3.2812136e-01,
      

In [ ]:
# Výpočet podobnosti (kosinova vzdalenost / podobnost) mezi slovy
embedding.wv.similarity("ekonomika", "stat")

0.9604165

## NLP
- ukážeme si jak použít NLP pro různé typy úloh
    - nebudeme tedy již vytvářet nový model, ale zaměříme se čistě na použití natrénovaného
    - použití natrénovaného modelu na reálných datech se říká inference
- doporučuji si projít stránku huggingface, obsahující spostu vytvořených modelů a datasetů (se skvělými filtry pro podrobné hledání)
- huggingface nabízí i kurz [NLP Course](https://huggingface.co/learn/nlp-course/chapter0/1) dostupný zdarma
    - kurz krásně popisuje jednotlivé témata NLP jak teoreticky tak i prakticky
    - navíc krásně funguje v ekosystému celé stránky, kde je možné stahovat natrénované modely i datasety pomocí knihovny [transformers](https://huggingface.co/docs/hub/transformers) a [datasets](https://huggingface.co/docs/hub/datasets-usage)

In [237]:
# Stáhneme si model bert-base-uncased – https://huggingface.co/google-bert/bert-base-uncased
# uncased: nerozlišuje mezi velkými a malými písmeny
# base: verze obsahující 110M parametrů
# naučen na Anglických datech
unmasker_model = pipeline('fill-mask', model='bert-base-uncased')

# V následujícím textu se snažíme doplnit slovo na pozici "[MASK]"
# Model nám vrátí predikce slov, které se na pozici hodí
unmasker_model("My favourite movie is The Lord of the [MASK].")

[{'score': 0.9915289878845215,
  'token': 7635,
  'token_str': 'rings',
  'sequence': 'my favourite movie is the lord of the rings.'},
 {'score': 0.003682296024635434,
  'token': 10029,
  'token_str': 'flies',
  'sequence': 'my favourite movie is the lord of the flies.'},
 {'score': 0.0005844107363373041,
  'token': 3614,
  'token_str': 'ring',
  'sequence': 'my favourite movie is the lord of the ring.'},
 {'score': 0.00044613334466703236,
  'token': 6841,
  'token_str': 'beast',
  'sequence': 'my favourite movie is the lord of the beast.'},
 {'score': 0.0002883195993490517,
  'token': 27754,
  'token_str': 'apes',
  'sequence': 'my favourite movie is the lord of the apes.'}]

In [238]:
# Použijeme model distilbert-NER – https://huggingface.co/dslim/distilbert-NER
# obsahuje 65.2M parametrů 
ner = pipeline("ner", model="dslim/distilbert-NER", aggregation_strategy="average")
ner("Bilbo Baggins celebrates his birthday and leaves the Ring to Frodo, his heir. Gandalf (a wizard) suspects it is a Ring of Power; seventeen years later, he confirms it was lost by the Dark Lord Sauron and counsels Frodo to take it away from the Shire.")[:5]

[{'entity_group': 'PER',
  'score': np.float32(0.99807286),
  'word': 'Bilbo Baggins',
  'start': np.int32(0),
  'end': np.int32(13)},
 {'entity_group': 'ORG',
  'score': np.float32(0.63286144),
  'word': 'Ring',
  'start': np.int32(53),
  'end': np.int32(57)},
 {'entity_group': 'PER',
  'score': np.float32(0.9991155),
  'word': 'Frodo',
  'start': np.int32(61),
  'end': np.int32(66)},
 {'entity_group': 'PER',
  'score': np.float32(0.99817365),
  'word': 'Gandalf',
  'start': np.int32(78),
  'end': np.int32(85)},
 {'entity_group': 'ORG',
  'score': np.float32(0.7092921),
  'word': 'Ring of Power',
  'start': np.int32(114),
  'end': np.int32(127)}]

In [239]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Who is Gandalf?",
    context="""Gandalf is a protagonist in J. R. R. Tolkien's novels The Hobbit and The Lord of the Rings. He is a wizard, one of the Istari order, and the leader of the Company of the Ring. Tolkien took the name "Gandalf" from the Old Norse "Catalogue of Dwarves" (Dvergatal) in the Völuspá.""",
)

{'score': 0.2236422449350357, 'start': 98, 'end': 106, 'answer': 'a wizard'}